# Neo4j

Nodes:
- Product
    - productID
    - name
    - price
    - brand

Relations:
- Product -[:ALSO_BOUGHT]- Product   
- Product -[:ALSO_VIEWED]- Product
- Product -[:ALSO_BOUGHT]- Product
- Product -[:BUY_AFTER_VIEWING]-Product
    
Steps:
1. Populate the DB with all products
2. Create the relationships

In [17]:
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "amazon"), encryption=False)
session = driver.session()

### Step 1: Products
(We do not include in the graph the products without a title and/or without related products)

In [18]:
import time 
import datetime

def parse_line(line):
    product = eval(line)
    params = {}
    if (product.has_key('title') and product.has_key('related')):
        params['asin'] = product['asin']
        params['title'] = product['title']
        if product.has_key('price'):
            params['price'] = product['price']
        else:
            params['price'] = None
    return params

start = time.time()
#session.run("CREATE CONSTRAINT ON (p:Product) ASSERT p.productID IS UNIQUE")

with open("C:\\dtu\\ctbd\\amazon_dataset\\metadata_no_books.json", "r") as data:
    line_count=0
    products = []
    for line in data:
        products.append(parse_line(line))
        line_count = line_count + 1
        if(line_count % 5000 == 0):#every 1000 lines
            tx = session.begin_transaction()
            #with session.begin_transaction() as tx:
            for product in products:
                if not (product.has_key('title')):
                    continue
                else:
                    create_query="""CREATE (p:Product {productID:{asin}})
                    SET p.name={title}, p.price=toFloat({price})"""
                    res = tx.run(create_query, product)
                    for r in res:
                        continue
            tx.commit() # check every 1000 queries
            products = []
            if line_count % 60000 == 0: #every 300000 lines
                print '%.3f %% completed' % (line_count*1.0/60000 ,) # print every 5%
session.close()
end = time.time()
print "executed in:", datetime.timedelta(seconds = (end - start))

1.000 % completed
2.000 % completed
3.000 % completed
4.000 % completed
5.000 % completed
6.000 % completed
7.000 % completed
8.000 % completed
9.000 % completed
10.000 % completed
11.000 % completed
12.000 % completed
13.000 % completed
14.000 % completed
15.000 % completed
16.000 % completed
17.000 % completed
18.000 % completed
19.000 % completed
20.000 % completed
21.000 % completed
22.000 % completed
23.000 % completed
24.000 % completed
25.000 % completed
26.000 % completed
27.000 % completed
28.000 % completed
29.000 % completed
30.000 % completed
31.000 % completed
32.000 % completed
33.000 % completed
34.000 % completed
35.000 % completed
36.000 % completed
37.000 % completed
38.000 % completed
39.000 % completed
40.000 % completed
41.000 % completed
42.000 % completed
43.000 % completed
44.000 % completed
45.000 % completed
46.000 % completed
47.000 % completed
48.000 % completed
49.000 % completed
50.000 % completed
51.000 % completed
52.000 % completed
53.000 % completed
54

### Step 2: Relationships

In [ ]:
def parse_line(line):
    product = eval(line)
    params = {}
    if (product.has_key('title') and product.has_key('related')):
        params['asin'] = product['asin']
        params['title'] = product['title']
        for key in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if not product['related'].has_key(key):
                params[key] = []
            else:
                params[key] = product['related'][key]
    return params

with open("/Users/lucacambiaghi/amz_dataset/metadata_no_books.json", "r") as data:
    count=0
    for line in data:
        params = parse_line(line)
        if not (params.has_key('title')):
            count +=1
            continue
        create_query = """MATCH (p:Product),(q:Product)
        WHERE p.productID = {asin} AND q.productID IN {also_bought}
        CREATE UNIQUE (p)-[:ALSO_BOUGHT]-(q)
        
        MATCH (p:Product),(q:Product)
        WHERE p.productID = {asin} AND q.productID IN {also_viewed}
        CREATE UNIQUE (p)-[:ALSO_BOUGHT]-(q)
        
        MATCH (p:Product),(q:Product)
        WHERE p.productID = {asin} AND q.productID IN {bought_together}
        CREATE UNIQUE (p)-[:ALSO_BOUGHT]-(q)
        
        MATCH (p:Product),(q:Product)
        WHERE p.productID = {asin} AND q.productID IN {buy_after_viewing}
        CREATE UNIQUE (p)-[:ALSO_BOUGHT]-(q)"""
        session.run(create_query, params)
        count+=1
        if count % 600000 == 0:
            print str(count/600000) + '% completed'
            break
print "done"
session.close()

### Query
- 